# Preparing the Enviorment




## Config

In [3]:
import os
import shutil
import subprocess
import sys
from dataclasses import dataclass, field
from pathlib import Path
from typing import List, Literal

EnvModeType = Literal["colab", "colab-vm", "remote", "local"]


@dataclass(frozen=True)
class Config:
    env_mode: EnvModeType
    selected_subsets: List[str] = field(
        default_factory=lambda: ["SUDOUE-4", "SUDOUE-5", "SUDOUE-6"]
    )

    # Derived fields (init=False)
    data_root: Path = field(init=False)
    base_dir: Path = field(init=False)
    taco_raw_dir: Path = field(init=False)
    taco_file_paths: List[Path] = field(init=False)
    normalized_sets_dir: Path = field(init=False)
    finetune_dir: Path = field(init=False)
    train_dir: Path = field(init=False)
    val_dir: Path = field(init=False)
    test_dir: Path = field(init=False)

    def __post_init__(self) -> None:
        # Set data_root based on env_mode (defaults; override in factory if needed)
        data_root_map = {
            "local": Path("/mnt/shared"),
            "remote": Path.home(),
            "colab": Path("/content/drive/MyDrive"),
            "colab-vm": Path("/content/MyDrive"),  # only normalized sets are in vm
        }
        object.__setattr__(
            self, "data_root", data_root_map.get(self.env_mode, Path.cwd())
        )

        # Derive other paths
        object.__setattr__(self, "base_dir", self.data_root / "datasets/sen2venus")
        object.__setattr__(self, "taco_raw_dir", self.base_dir / "TACO_raw_data")
        object.__setattr__(
            self,
            "taco_file_paths",
            [self.taco_raw_dir / f"{subset}.taco" for subset in self.selected_subsets],
        )
        normalized_sets_dir_map = {
            "local": self.base_dir / "normalized_sets",
            "remote": self.base_dir / "normalized_sets",
            "colab": self.base_dir / "normalized_sets",
            "colab-vm": Path("/content/normalized_sets"),
        }
        object.__setattr__(
            self,
            "normalized_sets_dir",
            normalized_sets_dir_map.get(
                self.env_mode, self.base_dir / "normalized_sets"
            ),
        )
        object.__setattr__(self, "finetune_dir", self.base_dir / "finetune")
        object.__setattr__(self, "train_dir", self.normalized_sets_dir / "train")
        object.__setattr__(self, "val_dir", self.normalized_sets_dir / "val")
        object.__setattr__(self, "test_dir", self.normalized_sets_dir / "test")

    def validate(self) -> None:
        """Validate config paths exist; raise errors otherwise."""
        missing_paths = []
        for attr in [
            "data_root",
            "base_dir",
            "taco_raw_dir",
            "normalized_sets_dir",
            "finetune_dir",
            "train_dir",
            "val_dir",
            "test_dir",
        ]:
            path: Path = getattr(self, attr)
            if not path.exists():
                missing_paths.append(str(path))
        if missing_paths:
            raise ValueError(f"Missing paths: {', '.join(missing_paths)}")
        for file_path in self.taco_file_paths:
            if not file_path.exists():
                missing_paths.append(str(file_path))
        if missing_paths:
            raise ValueError(f"Missing taco files: {', '.join(missing_paths)}")


def setup_environment(env_mode: EnvModeType) -> None:
    """Perform environment-specific setup (side effects isolated here)."""
    if env_mode.startswith("colab"):
        try:
            # import super_image  # noqa: F401
            import rasterio  # noqa: F401
            import tacoreader  # noqa: F401
        except ImportError:
            print("Installing external packages...")
            try:
                subprocess.run(
                    [
                        "pip",
                        "install",
                        "--quiet",
                        # "super-image",
                        "rasterio",
                        "tacoreader",
                    ],
                    check=True,
                )
            except subprocess.CalledProcessError as e:
                raise RuntimeError(f"Failed to install package: {e}")

        try:
            from google.colab import drive

            drive.mount("/content/drive", force_remount=True)
        except ImportError:
            raise RuntimeError("Google Colab module not found. Are you in Colab?")
        except Exception as e:
            raise RuntimeError(f"Failed to mount Google Drive: {e}")

        try:
            print("Fetching IP info...")
            result = subprocess.run(
                ["curl", "ipinfo.io"], capture_output=True, text=True, check=True
            )
            print(f"IP Info: {result.stdout}")
        except subprocess.CalledProcessError as e:
            print(f"Warning: Failed to fetch IP info: {e}")

        # Optional: Copy data to local /content for faster I/O in Colab
        if env_mode.endswith("vm"):
            colab_vm_dir = Path("/content/taco_normalized")
            if not colab_vm_dir.exists():
                print(
                    "Copying normalized data to local Colab storage for performance..."
                )
                shutil.copytree(
                    Path("/content/drive/MyDrive/datasets/sen2venus/normalized_sets"),
                    colab_vm_dir,
                )
                print("Copy complete.")
            # Avoid os.chdir; let users handle working dir if needed

    elif env_mode == "remote":
        print("Remote environment detected. No specific setup needed.")

    elif env_mode == "local":
        print("Local environment detected. Ensuring dependencies...")


def create_config(env_mode: EnvModeType | None = None) -> Config:
    """Factory to create and setup config based on detected environment."""
    if env_mode is None:
        if "google.colab" in sys.modules:
            env_mode = "colab"
        elif "REMOTE_ENV_VAR" in os.environ:
            env_mode = "remote"
        else:
            env_mode = "local"

    setup_environment(env_mode)
    config = Config(env_mode=env_mode)
    config.validate()
    return config


In [4]:
config = create_config('colab')
print(config.data_root)

Mounted at /content/drive
Fetching IP info...
IP Info: {
  "ip": "34.27.37.33",
  "hostname": "33.37.27.34.bc.googleusercontent.com",
  "city": "Council Bluffs",
  "region": "Iowa",
  "country": "US",
  "loc": "41.2619,-95.8608",
  "org": "AS396982 Google LLC",
  "postal": "51502",
  "timezone": "America/Chicago",
  "readme": "https://ipinfo.io/missingauth"
}
/content/drive/MyDrive


## Project Imports

In [5]:
import torch
from torch.utils.data import Dataset, random_split, DataLoader
from torchvision.transforms import v2

from PIL import Image
import tacoreader
from tacoreader import TortillaDataFrame
import numpy as np
import rasterio as rio
import matplotlib.pyplot as plt
import pandas as pd
import pyarrow as pa
import multiprocessing
from tqdm import tqdm
import os
from typing import Dict, Callable, Union

## Paths and Directories

In [ ]:
from google.colab import drive
from pathlib import Path

drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
ROOT_PATH = Path('/content/drive/MyDrive/datasets/sen2venus')

TACO_RAW_DIR = ROOT_PATH / 'TACO_raw_data'
os.makedirs(TACO_RAW_DIR, exist_ok=True)
print(f"Data will be saved to: {TACO_RAW_DIR}")

SELECTED_SUBSETS = [
    "SUDOUE-4",
    "SUDOUE-5",
    "SUDOUE-6"
]
TACO_FILE_PATHS = [TACO_RAW_DIR / f"{site_name}.taco" for site_name in SELECTED_SUBSETS]


NORMALIZED_SETS_DIR = ROOT_PATH / 'normalized_sets'
os.makedirs(NORMALIZED_SETS_DIR, exist_ok=True)
print(f"Normalaized datest will be saved to: {NORMALIZED_SETS_DIR}")

TRAIN_SAVE_DIR = NORMALIZED_SETS_DIR / 'train'
os.makedirs(TRAIN_SAVE_DIR, exist_ok=True)
print(f"Train data will be saved to: {TRAIN_SAVE_DIR}")

VAL_SAVE_DIR = NORMALIZED_SETS_DIR / 'val'
os.makedirs(VAL_SAVE_DIR, exist_ok=True)
print(f"Validation data will be saved to: {VAL_SAVE_DIR}")

TEST_SAVE_DIR = NORMALIZED_SETS_DIR / 'test'
os.makedirs(TEST_SAVE_DIR, exist_ok=True)
print(f"Test data will be saved to: {TEST_SAVE_DIR}")

Data will be saved to: /content/drive/MyDrive/datasets/sen2venus/TACO_raw_data
Normalaized datest will be saved to: /content/drive/MyDrive/datasets/sen2venus/normalized_sets
Train data will be saved to: /content/drive/MyDrive/datasets/sen2venus/normalized_sets/train
Validation data will be saved to: /content/drive/MyDrive/datasets/sen2venus/normalized_sets/val
Test data will be saved to: /content/drive/MyDrive/datasets/sen2venus/normalized_sets/test


# Step 1: Download the Selected Regions (one time only)

In [ ]:
def get_taco_subset(site_name: str)->TortillaDataFrame:
  dataset = tacoreader.load("tacofoundation:sen2venus")

  # The 'region' column holds the site name, e.g., 'SUDOUE-4'
  site_df = dataset[dataset["region"] == site_name]

  if len(site_df) == 0:
    raise ValueError(f'invalid site name {site_name}')

  return site_df


def compile_save_subset(output_filename: str| Path, site_df: TortillaDataFrame)->None:
  print(f"Compiling {len(site_df)} patches into {output_filename}...")
  tacoreader.compile(dataframe=site_df, output=output_filename, nworkers=8)


def get_compile_subsets(site_names : List[str])->None:
  for site_name in site_names:
    print(f"\nProcessing site: {site_name}")
    site_df = get_taco_subset(site_name)
    output_filename = TACO_RAW_DIR / f"{site_name}.taco"
    output_filename = config.taco_raw_dir / f"{site_name}.taco"
    compile_save_subset(output_filename, site_df)


In [ ]:
SELECTED_SUBSETS = [
    # "SUDOUE-4",
    "SUDOUE-5",
    "SUDOUE-6"
]

get_compile_subsets(SELECTED_SUBSETS)


Processing site: SUDOUE-5
Compiling 2176 patches into /content/drive/MyDrive/datasets/sen2venus/TACO_raw_data/SUDOUE-5.taco...


Rolling out a tortilla: 100%|██████████| 2176/2176 [02:20<00:00, 15.48file/s]



Processing site: SUDOUE-6
Compiling 2435 patches into /content/drive/MyDrive/datasets/sen2venus/TACO_raw_data/SUDOUE-6.taco...


Grilling a tortilla 🫓: 100%|██████████| 2435/2435 [02:46<00:00, 14.62file/s]


# Step 2: Define PyTorch Datasets

In [6]:
class TACORGBDataset(Dataset):
    """
    Custom Dataset for reading compiled .taco files.
    """

    def __init__(
        self,
        taco_file_paths: List[str | Path],
        transform: Callable = None,
        lr_band_combo: List[int] = [1, 2, 3], # Blue, Green, Red of Sentinel-2
        hr_band_combo: List[int] = [1, 2, 3], # Blue, Green, Red of venus
    ):
        """
        Args:
            taco_file_paths (List[str]): A list of file paths to the .taco files.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.taco_files = [str(p) for p in taco_file_paths]
        self.transform = transform
        self.lr_band_combo = lr_band_combo
        self.hr_band_combo = hr_band_combo

        # Load and concatenate all specified .taco files into a single dataframe
        print(f"Loading data from: {self.taco_files}")
        # Convert PosixPath objects to strings
        list_of_dfs = [tacoreader.load(p) for p in self.taco_files]
        self.dataframe = pd.concat(list_of_dfs, ignore_index=True)
        print(f"Successfully loaded a total of {len(self.dataframe)} samples.")

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx) -> Dict[str, torch.Tensor]:
        if torch.is_tensor(idx):
            idx = idx.tolist()

        # Get the data for one row from the dataframe
        row = self.dataframe.read(idx)

        # Retrieve the data
        lr, hr = row.read(0), row.read(1)
        with rio.open(lr) as src_lr, rio.open(hr) as src_hr:
            lr_image_np = src_lr.read(self.lr_band_combo)
            hr_image_np = src_hr.read(self.hr_band_combo)

        # Convert directly to PyTorch Tensors. This preserves the uint16 data.
        lr_tensor = torch.from_numpy(lr_image_np)
        hr_tensor = torch.from_numpy(hr_image_np)

        lr_tensor = lr_tensor.float()
        hr_tensor = hr_tensor.float()

        sample = {"lr": lr_tensor, "hr": hr_tensor}

        if self.transform:
            sample = self.transform(sample)

        return sample


class NormalizedTacoDataset(Dataset):
    """
    Custom Wrapper Datasetclass for applying normalization
    on Tran/Test/Validation subsets.
    """

    def __init__(self, subset, transform):
        self.subset = subset
        self.transform = transform

    def __getitem__(self, index):
        # Gets the raw tensor from the subset
        sample = self.subset[index]
        # Applies the final normalization transform
        return self.transform(sample)

    def __len__(self):
        return len(self.subset)


class PreNormalizedDataset(Dataset):
    """
    An efficient Dataset that reads pre-processed, sharded tensor files.
    """

    def __init__(self, shard_dir: Union[str, Path]):
        """
        Args:
            shard_dir (Union[str, Path]): The directory containing the shard files.
        """
        # Immediately convert the input to a Path object to standardize its type.
        self.shard_dir = Path(shard_dir)

        # Use the object-oriented .glob() method to find all .pt files.
        # It returns a generator, so we convert to a list before sorting.
        self.shard_paths: List[Path] = sorted(self.shard_dir.glob("*.pt"))

        if not self.shard_paths:
            raise ValueError(f"No shard files ('*.pt') found in {self.shard_dir}")

        # The rest of the logic remains the same, but is more robust
        # because torch.load() works seamlessly with Path objects.

        # Load the first shard to determine the shard_size
        first_shard = torch.load(self.shard_paths[0])
        self.shard_size = len(first_shard)

        # Load the last shard to calculate the total length accurately
        last_shard = torch.load(self.shard_paths[-1])
        self.length = (len(self.shard_paths) - 1) * self.shard_size + len(last_shard)

        # A simple cache to hold the most recently used shard
        self._cache = {}
        # A variable to keep track of which shard is currently in the cache
        self._cached_shard_index = -1

        print(
            f"Initialized dataset from {self.shard_dir} with {self.length} samples across {len(self.shard_paths)} shards."
        )

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        shard_index = idx // self.shard_size
        index_in_shard = idx % self.shard_size

        # Check if the required shard is currently cached
        if shard_index != self._cached_shard_index:
            # If not, load the correct shard from disk and update the cache
            shard_path = self.shard_paths[shard_index]
            self._cache = torch.load(shard_path)
            self._cached_shard_index = shard_index

        # coupled with TACORGBDataset dataset class
        # each item in the shard is a squeezed dictionary with keys lr and hr
        squeezed_sample = self._cache[index_in_shard]
        return squeezed_sample["lr"], squeezed_sample["hr"]


In [7]:
full_dataset = TACORGBDataset(taco_file_paths=config.taco_file_paths)

Loading data from: ['/content/drive/MyDrive/datasets/sen2venus/TACO_raw_data/SUDOUE-4.taco', '/content/drive/MyDrive/datasets/sen2venus/TACO_raw_data/SUDOUE-5.taco', '/content/drive/MyDrive/datasets/sen2venus/TACO_raw_data/SUDOUE-6.taco']
Successfully loaded a total of 5546 samples.


In [8]:
first_item = full_dataset[101]
print(f"LR image tensor shape: {first_item['lr'].shape}")
print(f"HR image tensor shape: {first_item['hr'].shape}")

LR image tensor shape: torch.Size([3, 128, 128])
HR image tensor shape: torch.Size([3, 256, 256])


# Step 3: Split the Dataset 80/10/10

In [9]:
# Define split sizes
train_size = int(0.8 * len(full_dataset))
val_size = int(0.1 * len(full_dataset))
test_size = len(full_dataset) - train_size - val_size

# Perform the split
train_subset, val_subset, test_subset = random_split(full_dataset, [train_size, val_size, test_size])

print(f"Train samples: {len(train_subset)}, Validation samples: {len(val_subset)}, Test samples: {len(test_subset)}")

Train samples: 4436, Validation samples: 554, Test samples: 556


# Step 4: Confirm the Preprocessing

> Several preprocessing steps have already been applied to the dataset by `TACO foundation`:

- **Handling of negative pixel values:** Some Sentinel-2 images contained negative pixel values, which have been masked to **0**.
- **No-data values:** All no-data values are now set to **65525** for consistency.
- **Data type conversion:** The original dataset was stored in **int16**, and it has been converted to **uint16** for storage and processing.

# Step 5: Pre-Normalization Statistical Analysis

In [18]:
def get_subset_stats(loader: DataLoader):
    """
    Calculates statistics by iterating through a DataLoader, processing one batch at a time.
    This is significantly more efficient than loading all samples into memory first.
    """

    # Initialize trackers for the statistics we need
    # For min/max, we can track the running value.
    # For mean/std, the correct way to aggregate across batches is to track sums.

    # Initialize min/max trackers for 3 channels (R, G, B)
    lr_max = torch.tensor([-float('inf')] * 3)
    hr_max = torch.tensor([-float('inf')] * 3)
    lr_min = torch.tensor([float('inf')] * 3)
    hr_min = torch.tensor([float('inf')] * 3)

    # Trackers for a numerically stable standard deviation calculation
    lr_sum = torch.zeros(3)
    hr_sum = torch.zeros(3)
    lr_sum_sq = torch.zeros(3)
    hr_sum_sq = torch.zeros(3)
    pixel_count = 0

    # Iterate through the batches provided by the DataLoader
    for batch in tqdm(loader, desc="Calculating Stats"):
        # Handle both dictionary (from TACORGBDataset) and tuple (from PreNormalizedDataset) outputs
        if isinstance(batch, dict):
            lr_batch = batch['lr'] # Shape: (B, C, H, W) e.g., (64, 3, 128, 128)
            hr_batch = batch['hr']
        elif isinstance(batch, (list, tuple)):
            lr_batch, hr_batch = batch
        else:
            raise TypeError(f"Unexpected batch type: {type(batch)}")


        # --- Min/Max Calculation (Batch-wise) ---
        # Find the max value along all dims except the channel dim
        current_lr_max = torch.amax(lr_batch, dim=(0, 2, 3))
        current_hr_max = torch.amax(hr_batch, dim=(0, 2, 3))
        # Update our running global max
        lr_max = torch.maximum(lr_max, current_lr_max)
        hr_max = torch.maximum(hr_max, current_hr_max)

        # Do the same for min
        current_lr_min = torch.amin(lr_batch, dim=(0, 2, 3))
        current_hr_min = torch.amin(hr_batch, dim=(0, 2, 3))
        lr_min = torch.minimum(lr_min, current_lr_min)
        hr_min = torch.minimum(hr_min, current_hr_min)

        # --- Mean/Std Calculation (Batch-wise) ---
        # Sum up all values per channel for this batch
        lr_sum += torch.sum(lr_batch, dim=(0, 2, 3))
        hr_sum += torch.sum(hr_batch, dim=(0, 2, 3))
        # Sum up the squares of all values per channel
        lr_sum_sq += torch.sum(lr_batch**2, dim=(0, 2, 3))
        hr_sum_sq += torch.sum(hr_batch**2, dim=(0, 2, 3))

        # Count the total number of pixels per channel
        # Batch size * height * width
        pixel_count += lr_batch.shape[0] * lr_batch.shape[2] * lr_batch.shape[3]

    # Finalize the mean and standard deviation calculations
    mean_lr = lr_sum / pixel_count
    mean_hr = hr_sum / pixel_count

    # std_lr = torch.sqrt((lr_sum_sq / pixel_count) - mean_lr**2)
    # std_hr = torch.sqrt((hr_sum_sq / pixel_count) - mean_hr**2)

    # Inside get_stats_efficiently, before the return statement:
    variance_lr = (lr_sum_sq / pixel_count) - mean_lr**2
    variance_hr = (hr_sum_sq / pixel_count) - mean_hr**2

    # Clamp the variance at 0 to prevent negative values from causing nans in the sqrt.
    std_lr = torch.sqrt(torch.clamp(variance_lr, min=0))
    std_hr = torch.sqrt(torch.clamp(variance_hr, min=0))

    # Compile results into a dictionary
    stats = {
        'lr_max_per_channel': lr_max,
        'hr_max_per_channel': hr_max,
        'lr_min_per_channel': lr_min,
        'hr_min_per_channel': hr_min,
        'lr_mean_per_channel': mean_lr,
        'hr_mean_per_channel': mean_hr,
        'lr_std_per_channel': std_lr,
        'hr_std_per_channel': std_hr,
    }

    return stats

In [28]:
def pivot_stats_df(stats_df: pd.DataFrame)->pd.DataFrame:
  stats_df['Color'] = ['B', 'G', 'R']
  df_melted = stats_df.melt(id_vars='Color', var_name='Metric', value_name='Value')
  pivoted_df = df_melted.pivot(index='Metric', columns='Color', values='Value')

  return pivoted_df

## Dataloaders

In [11]:
num_workers = 2
batch_size = 64

### Train Stats

In [25]:
train_stats_loader = DataLoader(
    train_subset,
    batch_size=batch_size,
    num_workers=num_workers,
    shuffle=False  # No need to shuffle for statistical analysis
)

train_stats = get_subset_stats(train_stats_loader)

train_stats = {
    key: value.tolist() for key, value in train_stats.items()
}
train_stats_pre_df = pd.DataFrame(train_stats)
display(train_stats_pre_df)

Calculating Stats: 100%|██████████| 70/70 [03:33<00:00,  3.06s/it]


,lr_max_per_channel,hr_max_per_channel,lr_min_per_channel,hr_min_per_channel,lr_mean_per_channel,hr_mean_per_channel,lr_std_per_channel,hr_std_per_channel
0,8683.0,11557.0,0.0,0.0,338.281494,1369.927856,220.427521,0.0
1,9235.0,11554.0,0.0,0.0,576.025146,2322.658447,265.718842,0.0
2,12752.0,12518.0,0.0,0.0,548.783508,2213.542725,378.689240,0.0


In [29]:
display(pivot_stats_df(train_stats_pre_df))

Color,B,G,R
Metric,,,
hr_max_per_channel,11557.000000,11554.000000,12518.000000
hr_mean_per_channel,1369.927856,2322.658447,2213.542725
hr_min_per_channel,0.000000,0.000000,0.000000
hr_std_per_channel,0.000000,0.000000,0.000000
lr_max_per_channel,8683.000000,9235.000000,12752.000000
lr_mean_per_channel,338.281494,576.025146,548.783508
lr_min_per_channel,0.000000,0.000000,0.000000
lr_std_per_channel,220.427521,265.718842,378.689240


### Validation Stats

In [23]:
val_stats_loader = DataLoader(
    val_subset,
    batch_size=batch_size,
    num_workers=num_workers,
    shuffle=False  # No need to shuffle for statistical analysis
)

val_stats = get_subset_stats(val_stats_loader)

val_stats = {
    key: value.tolist() for key, value in val_stats.items()
}
val_stats_pre_df = pd.DataFrame(val_stats)
display(val_stats_pre_df)

Calculating Stats: 100%|██████████| 9/9 [00:31<00:00,  3.53s/it]


,lr_max_per_channel,hr_max_per_channel,lr_min_per_channel,hr_min_per_channel,lr_mean_per_channel,hr_mean_per_channel,lr_std_per_channel,hr_std_per_channel
0,10978.0,6214.0,0.0,0.0,332.809906,1347.939941,214.497620,0.0
1,8144.0,6910.0,0.0,0.0,567.810303,2290.613525,261.039520,0.0
2,7125.0,8261.0,0.0,0.0,539.888367,2182.103027,372.532745,0.0


In [30]:
display(pivot_stats_df(val_stats_pre_df))

Color,B,G,R
Metric,,,
hr_max_per_channel,6214.000000,6910.000000,8261.000000
hr_mean_per_channel,1347.939941,2290.613525,2182.103027
hr_min_per_channel,0.000000,0.000000,0.000000
hr_std_per_channel,0.000000,0.000000,0.000000
lr_max_per_channel,10978.000000,8144.000000,7125.000000
lr_mean_per_channel,332.809906,567.810303,539.888367
lr_min_per_channel,0.000000,0.000000,0.000000
lr_std_per_channel,214.497620,261.039520,372.532745


### Test Stats

In [22]:
test_stats_loader = DataLoader(
    test_subset,
    batch_size=batch_size,
    num_workers=num_workers,
    shuffle=False  # No need to shuffle for statistical analysis
)

test_stats = get_subset_stats(test_stats_loader)

test_stats = {
    key: value.tolist() for key, value in test_stats.items()
}
test_stats_pre_df = pd.DataFrame(test_stats)
display(test_stats_pre_df)

Calculating Stats: 100%|██████████| 9/9 [00:28<00:00,  3.15s/it]


,lr_max_per_channel,hr_max_per_channel,lr_min_per_channel,hr_min_per_channel,lr_mean_per_channel,hr_mean_per_channel,lr_std_per_channel,hr_std_per_channel
0,6545.0,6931.0,0.0,0.0,322.960846,1310.386597,212.768997,0.0
1,7311.0,6911.0,0.0,0.0,559.530884,2257.349609,257.391113,0.0
2,6930.0,6830.0,0.0,0.0,524.640808,2118.355469,367.924500,0.0


In [31]:
display(pivot_stats_df(test_stats_pre_df))

Color,B,G,R
Metric,,,
hr_max_per_channel,6931.000000,6911.000000,6830.000000
hr_mean_per_channel,1310.386597,2257.349609,2118.355469
hr_min_per_channel,0.000000,0.000000,0.000000
hr_std_per_channel,0.000000,0.000000,0.000000
lr_max_per_channel,6545.000000,7311.000000,6930.000000
lr_mean_per_channel,322.960846,559.530884,524.640808
lr_min_per_channel,0.000000,0.000000,0.000000
lr_std_per_channel,212.768997,257.391113,367.924500


# Step 6: Subset Normalization (MinMax Scaling)

In [ ]:
def normalize_tensor_sample(sample: Dict[str, torch.Tensor],
                               lr_max_per_channel: torch.Tensor,
                               hr_max_per_channel: torch.Tensor) -> Dict[str, torch.Tensor]:
    """
    Normalizes a sample dictionary of multi-channel float tensors
    using per-channel maximum values.

    Args:
        sample (Dict[str, torch.Tensor]): The input sample, e.g., {'lr': tensor, 'hr': tensor}.
        lr_max_per_channel (torch.Tensor): A 1D tensor of shape (3,) containing the max values for the R, G, B channels of the LR image.
        hr_max_per_channel (torch.Tensor): A 1D tensor of shape (3,) for the HR image.

    Returns:
        Dict[str, torch.Tensor]: The normalized sample.
    """
    # Reshape the max-value tensors from (3,) to (3, 1, 1) for broadcasting.
    # This tells PyTorch to align the channel dimension and "stretch" the other two.
    lr_max_reshaped = lr_max_per_channel.view(3, 1, 1)
    hr_max_reshaped = hr_max_per_channel.view(3, 1, 1)

    # Perform the per-channel division.
    # PyTorch will broadcast the (3, 1, 1) tensor across the (3, H, W) image tensor.
    # This divides each channel by its specific max value efficiently.
    lr = sample['lr'] / lr_max_reshaped
    hr = sample['hr'] / hr_max_reshaped

    # Clamp values to ensure they are strictly within [0,1]
    return {'lr': torch.clamp(lr, 0.0, 1.0), 'hr': torch.clamp(hr, 0.0, 1.0)}


In [ ]:
lr_max_per_channel = [8683.0, 9235.0, 11554.0] # according to train set
hr_max_per_channel = [11557.0, 11554.0, 12518.0] # according to train set

train_lr_maxes = torch.tensor(lr_max_per_channel, dtype=torch.float32)
train_hr_maxes = torch.tensor(hr_max_per_channel, dtype=torch.float32)

# Using a lambda to pass  calculated tensors to the normalization function
final_transform = lambda sample: normalize_tensor_sample(sample,
                                                            lr_max_per_channel=train_lr_maxes,
                                                            hr_max_per_channel=train_hr_maxes)


train_dataset = NormalizedTacoDataset(train_subset, final_transform)
val_dataset = NormalizedTacoDataset(val_subset, final_transform)
test_dataset = NormalizedTacoDataset(test_subset, final_transform)

In [ ]:
sample_to_check = train_dataset[0]
lr_check = sample_to_check['lr']
print(f"\nMax value in final normalized LR Red channel: {lr_check[0].max():.4f}")   # Should be <= 1.0
print(f"Max value in final normalized LR Green channel: {lr_check[1].max():.4f}") # Should be <= 1.0
print(f"Max value in final normalized LR Blue channel: {lr_check[2].max():.4f}")  # Should be <= 1.0


Max value in final normalized LR Red channel: 0.0955
Max value in final normalized LR Green channel: 0.1449
Max value in final normalized LR Blue channel: 0.1431


# Step 7: Save Normalized Subsets

In [ ]:
def save_processed_dataset(dataset: Dataset, save_dir: str, shard_size: int = 1000):
    """
    Iterates through a dataset and saves the processed tensors in sharded .pt files.

    Args:
        dataset (Dataset): The dataset to save (e.g.,train_dataset).
        save_dir (str): The directory  to save the shards.
        shard_size (int): The number of samples to store in each shard file.
    """
    os.makedirs(save_dir, exist_ok=True)

    # Use a DataLoader for efficient reading from the source dataset
    loader = DataLoader(dataset, batch_size=1, num_workers=2)

    shard = []
    shard_index = 0

    for i, sample in enumerate(tqdm(loader, desc=f"Saving to {os.path.basename(save_dir)}")):
        # The loader adds an extra batch dimension, so we squeeze it out.
        # We also move tensors to CPU to ensure they can be saved from any device.
        squeezed_sample = {key: val.squeeze(0).cpu() for key, val in sample.items()}
        shard.append(squeezed_sample)

        # When the shard is full, or it's the last sample, save it
        if len(shard) == shard_size or i == len(loader) - 1:
            shard_path = os.path.join(save_dir, f"shard_{shard_index}.pt")
            torch.save(shard, shard_path)

            # Reset for the next shard
            shard = []
            shard_index += 1

    print(f"\nSuccessfully saved {i+1} samples in {shard_index} shards to {save_dir}")

In [ ]:
print("--- Saving Training Set ---")
save_processed_dataset(train_dataset, TRAIN_SAVE_DIR)

print("\n--- Saving Validation Set ---")
save_processed_dataset(val_dataset, VAL_SAVE_DIR)

print("\n--- Saving Test Set ---")
save_processed_dataset(test_dataset, TEST_SAVE_DIR)

--- Saving Training Set ---


Saving to train: 100%|██████████| 4436/4436 [23:03<00:00,  3.21it/s]



Successfully saved 4436 samples in 5 shards to /content/drive/MyDrive/datasets/sen2venus/normalized_sets/train

--- Saving Validation Set ---


Saving to val: 100%|██████████| 554/554 [01:16<00:00,  7.28it/s]



Successfully saved 554 samples in 1 shards to /content/drive/MyDrive/datasets/sen2venus/normalized_sets/val

--- Saving Test Set ---


Saving to test: 100%|██████████| 556/556 [00:59<00:00,  9.38it/s]


Successfully saved 556 samples in 1 shards to /content/drive/MyDrive/datasets/sen2venus/normalized_sets/test


# Step 8: Post Normalization Statistical Analysis

In [15]:
final_train_dataset = PreNormalizedDataset(config.train_dir)
final_val_dataset = PreNormalizedDataset(config.val_dir)
final_test_dataset = PreNormalizedDataset(config.test_dir)

print(f"Loaded {len(final_train_dataset)} training samples.")
print(f"Loaded {len(final_val_dataset)} validation samples.")
print(f"Loaded {len(final_test_dataset)} test samples.")

Initialized dataset from /content/drive/MyDrive/datasets/sen2venus/normalized_sets/train with 4436 samples across 5 shards.
Initialized dataset from /content/drive/MyDrive/datasets/sen2venus/normalized_sets/val with 554 samples across 1 shards.
Initialized dataset from /content/drive/MyDrive/datasets/sen2venus/normalized_sets/test with 556 samples across 1 shards.
Loaded 4436 training samples.
Loaded 554 validation samples.
Loaded 556 test samples.


In [ ]:
batch_size = 64
num_workers = 2

In [21]:
print("\n--- Calculating Post-Normalization Stats for Training Set ---")
post_norm_train_loader = DataLoader(final_train_dataset, batch_size=batch_size, num_workers=num_workers)
train_stats_post = get_subset_stats(post_norm_train_loader)

train_stats_post = {
    key: value.tolist() for key, value in train_stats_post.items()
}
train_stats_df = pd.DataFrame(train_stats_post)
display(train_stats_df)


--- Calculating Post-Normalization Stats for Training Set ---


Calculating Stats: 100%|██████████| 70/70 [01:45<00:00,  1.50s/it]


,lr_max_per_channel,hr_max_per_channel,lr_min_per_channel,hr_min_per_channel,lr_mean_per_channel,hr_mean_per_channel,lr_std_per_channel,hr_std_per_channel
0,1.0,1.0,0.0,0.0,0.038670,0.117750,0.025273,0.0
1,1.0,1.0,0.0,0.0,0.062028,0.200009,0.028677,0.0
2,1.0,1.0,0.0,0.0,0.047128,0.175574,0.032641,0.0


In [32]:
display(pivot_stats_df(train_stats_df))

Color,B,G,R
Metric,,,
hr_max_per_channel,1.000000,1.000000,1.000000
hr_mean_per_channel,0.117750,0.200009,0.175574
hr_min_per_channel,0.000000,0.000000,0.000000
hr_std_per_channel,0.000000,0.000000,0.000000
lr_max_per_channel,1.000000,1.000000,1.000000
lr_mean_per_channel,0.038670,0.062028,0.047128
lr_min_per_channel,0.000000,0.000000,0.000000
lr_std_per_channel,0.025273,0.028677,0.032641


In [19]:
print("\n--- Calculating Post-Normalization Stats for Validation Set ---")
post_norm_val_loader = DataLoader(final_val_dataset, batch_size=batch_size, num_workers=num_workers)
val_stats_post = get_subset_stats(post_norm_val_loader)

val_stats_post = {
    key: value.tolist() for key, value in val_stats_post.items()
}
val_stats_df = pd.DataFrame(val_stats_post)
display(val_stats_df)


--- Calculating Post-Normalization Stats for Validation Set ---


Calculating Stats: 100%|██████████| 9/9 [00:04<00:00,  2.24it/s]


,lr_max_per_channel,hr_max_per_channel,lr_min_per_channel,hr_min_per_channel,lr_mean_per_channel,hr_mean_per_channel,lr_std_per_channel,hr_std_per_channel
0,0.808476,0.599723,0.0,0.0,0.039560,0.119222,0.025244,0.0
1,0.748890,0.598148,0.0,0.0,0.062966,0.201798,0.028544,0.0
2,0.586290,0.545614,0.0,0.0,0.048274,0.178841,0.032653,0.0


In [33]:
display(pivot_stats_df(val_stats_df))

Color,B,G,R
Metric,,,
hr_max_per_channel,0.599723,0.598148,0.545614
hr_mean_per_channel,0.119222,0.201798,0.178841
hr_min_per_channel,0.000000,0.000000,0.000000
hr_std_per_channel,0.000000,0.000000,0.000000
lr_max_per_channel,0.808476,0.748890,0.586290
lr_mean_per_channel,0.039560,0.062966,0.048274
lr_min_per_channel,0.000000,0.000000,0.000000
lr_std_per_channel,0.025244,0.028544,0.032653


In [20]:
print("\n--- Calculating Post-Normalization Stats for Test Set ---")
post_norm_test_loader = DataLoader(final_test_dataset, batch_size=batch_size, num_workers=num_workers)
test_stats_post = get_subset_stats(post_norm_test_loader)

test_stats_post = {
    key: value.tolist() for key, value in test_stats_post.items()
}
test_stats_df = pd.DataFrame(test_stats_post)
display(val_stats_df)


--- Calculating Post-Normalization Stats for Test Set ---


Calculating Stats: 100%|██████████| 9/9 [00:06<00:00,  1.49it/s]


,lr_max_per_channel,hr_max_per_channel,lr_min_per_channel,hr_min_per_channel,lr_mean_per_channel,hr_mean_per_channel,lr_std_per_channel,hr_std_per_channel
0,0.808476,0.599723,0.0,0.0,0.039560,0.119222,0.025244,0.0
1,0.748890,0.598148,0.0,0.0,0.062966,0.201798,0.028544,0.0
2,0.586290,0.545614,0.0,0.0,0.048274,0.178841,0.032653,0.0


In [34]:
display(pivot_stats_df(test_stats_df))

Color,B,G,R
Metric,,,
hr_max_per_channel,0.495630,0.598061,0.740134
hr_mean_per_channel,0.117081,0.199960,0.174729
hr_min_per_channel,0.000000,0.000000,0.000000
hr_std_per_channel,0.000000,0.000000,0.000000
lr_max_per_channel,0.784061,0.744775,0.654752
lr_mean_per_channel,0.038276,0.061876,0.046815
lr_min_per_channel,0.000000,0.000000,0.000000
lr_std_per_channel,0.024924,0.028414,0.032565
